### Load Library:

In [1]:
import numpy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

### Dataset Preparation:

In [ ]:
data = pd.read_csv('nci60_data.csv') # load dataset

instances = data.iloc[:-1, 1:-1] # get instances
labels = data.iloc[:-1, -1] # get labels

### Traditional Manifold:

### Modeling Manifold: